In [ ]:
!pip install selenium
import requests
import pickle
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import numpy as np
import math
pd.set_option('display.max_columns', None)

s = {}
years = np.arange(2010, 2016,1)
data = pd.DataFrame(columns = ['year','Team','OTeam','Score','OScore'])
url = 'https://www.basketball-reference.com/leagues/NBA_2011_games.html'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')
bs = soup.find_all('td',class_ = 'center')

In [ ]:
def get_player_match(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    away = {}
    home = {}
    drop_col = []

    w = 0
    for t in soup.find_all(['span','div','td'],class_ = ['section_anchor','overthrow table_container']): 
        if 'data-label="" id="box-' in str(t):
            team = str(t).partition('id="box-')[-1].partition('-')[0]
            if w == 0:
                if len(away) != 0:
                    w = 1
                    away['away'] = team
            else:
                if len(home) != 0:
                    w = 0
                    home['home'] = team

        b = BeautifulSoup(str(t), 'html.parser')
        for j in b.find_all(['tr']):  
            if 'href' in str(j):
                b = BeautifulSoup(str(j),'html.parser')
                for k in b.find_all(['th','td']):
    #                 if k['data-stat'] == 'reason':
    #                     if name in dict_.keys():
    #                         dict_.pop(name)
                    if k['data-stat'] == 'player':
                        if w == 0:
                            if k.text not in home.keys():

                                away[k.text] = k.text
                        if w == 1:
                            if k.text not in away.keys():
                                home[k.text] = k.text

                        name = k.text

    #                 if i['data-stat'] == 'player' and 'href' in str(i):
    #                     print(i)
                    if k['data-stat'] == 'reason':
    #                     print(k, len(k.text))
                        if len(k.text) > 1:
                            if name not in drop_col:
                                drop_col.append(name)
    for n in drop_col:
        if n in away:
            away.pop(n)
        else:
            home.pop(n)
    away = pd.DataFrame(away, index = [0]).T.reset_index(drop = True).T
    for i in away.columns:
        if i +1 == len(away.columns):
            away = away.rename(columns = {i:'team'})
        else:
            away = away.rename(columns = {i:'player_' + str(i)})    

    home = pd.DataFrame(home, index = [0]).T.reset_index(drop = True).T
    for i in home.columns:
        if i + 1 == len(home.columns):
            home = home.rename(columns = {i:'team'})
        else:
            home = home.rename(columns = {i:'player_' + str(i)})    
        
#         home = home.rename(columns = {i:'player_' + str(i)})  
    return away, home
def get_extend(df, idx):
    idx = 'player_' + str(idx)
    df[idx] = 0
    return df

def get_check_different(df,df1):
    num = df.shape[-1] - df1.shape[-1]
    
    df_idx = len(df.columns) 
    df1_idx = len(df1.columns)
    
    if num != 0:
        for _ in range(num):
            if df.shape[-1] < df1.shape[-1]:
                df = get_extend(df, df_idx) 
                df_idx += 1
            else:
                df1 = get_extend(df1,df1_idx)
                df1_idx += 1
        return df.fillna(0),df1.fillna(0)
    else:
        return df.fillna(0),df1.fillna(0)
    

In [ ]:
url = 'https://www.basketball-reference.com/leagues/NBA_2011_games.html'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')


month_name = soup.find_all(['div'], {'class': 'filter'})
month = []
for i in month_name:
    for m in i:
        if 'href' in str(m):
            month.append(str(m).partition('html">')[-1].partition('</a')[0])
d = pd.DataFrame()
# year = 2010
w = 0

for year in years:
#     d = pd.DataFrame()
#     w = 0

    for m in month:
        url = 'https://www.basketball-reference.com/leagues/NBA_{}_games-{}.html'.format(year,m.lower())
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'html.parser')
        bs = soup.find_all('td',class_ = 'center')    
        for b in bs: 
            if 'box_score_text' in str(b):
                url = 'https://www.basketball-reference.com/' + b.a['href']
                away, home = get_player_match(url)
                id_  = b.a['href'].partition('s/')[-1].partition('.html')[0]
#                 basic_d['month'] = m

            away, home = get_check_different(away,home)

            away_home = pd.concat([away,home], axis = 0)
            away_home['year'] = year
            away_home['month'] = m
            away_home['id'] = id_
            if w!= 0:
                away_home,d = get_check_different(away_home,d)
#                 display(d)
                
  

            if 'year' in away_home:
                pop_year = away_home.pop('year')
                pop_team = away_home.pop('month')
                pop_id =away_home.pop('id')
            
                away_home['year'] = pop_year
                away_home['month'] = pop_team
                away_home['id'] = pop_id
            if 'year' in d:
                pop_year = d.pop('year')
                pop_team = d.pop('month')
                pop_id = d.pop('id')

                d['year'] = pop_year
                d['month'] = pop_team
                d['id'] = pop_id

            d = pd.concat([d,away_home])
            w += 1
        print(d.shape)
    d.to_csv('d_{}.csv'.format(year),index = False)
    print('saved')